## IMPORTS

In [154]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [155]:
# loading the dataset to a pandas DataFrame
credit_card_data = pd.read_csv('Sample_Data/creditcard.csv')

In [156]:
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [157]:
credit_card_data.head(5)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [158]:
credit_card_data.tail(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,-1.593105,2.711941,-0.689256,4.626942,-0.924459,1.107641,1.991691,0.510632,-0.682920,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,-0.150189,0.915802,1.214756,-0.675143,1.164931,-0.711757,-0.025693,-1.221179,-1.545556,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,0.411614,0.063119,-0.183699,-0.510602,1.329284,0.140716,0.313502,0.395652,-0.577252,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,-1.933849,-0.962886,-1.042082,0.449624,1.962563,-0.608577,0.509928,1.113981,2.897849,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0
284806,172792.0,-0.533413,-0.189733,0.703337,-0.506271,-0.012546,-0.649617,1.577006,-0.414650,0.486180,-0.915427,-1.040458,-0.031513,-0.188093,-0.084316,0.041333,-0.302620,-0.660377,0.167430,-0.256117,0.382948,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,217.00,0


In [159]:
# data set features are encrypted by principal component analysis
# amount is in us dollars
# if the class is 1 then it is a fraudulent transaction else it is not
# 2 days transaction data

In [160]:
# data set information
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [161]:
# checking the number of missing values in each column
credit_card_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [162]:
# show the transaction class column
credit_card_data['Class'].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

In [163]:
# 492 frauds out of 284,807 transactions
# the dataset is highly unbalanced

--> handling the unbalanced data

In [164]:

# separating the data for analysis
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]

In [165]:
legit.shape

(284315, 31)

In [166]:
fraud.shape

(492, 31)

In [ ]:
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [168]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [169]:
# comparing the values for both transactions
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,0.009824,-0.006576,0.010832,0.000189,0.012064,0.000161,0.007164,0.011535,0.003887,-0.001178,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [170]:
# under-sampling
# build a sample dataset containing similar distribution of normal transactions and fraudulent transactions
# number of fraud transactions
legit_sample = legit.sample(n=492)


In [171]:
# concatenating two dataframes
new_dataset = pd.concat([legit_sample, fraud], axis=0)

In [172]:
new_dataset

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
65874,51782.0,0.985125,-1.735421,-0.347965,-1.194368,-1.362408,-0.937011,-0.150100,-0.414604,-2.173645,1.295732,-0.646338,-0.786700,0.765736,-0.090143,0.398949,-0.438219,0.486537,-0.425044,0.073022,0.298054,-0.404756,-1.506348,-0.100852,-0.015105,0.182456,-0.539072,-0.046081,0.068398,313.43,0
255857,157434.0,-1.521088,-1.708103,0.906377,-2.766646,0.234871,0.579226,-0.065909,0.161328,-1.851499,0.432489,-0.708131,-0.537107,-0.147410,-0.848400,-3.137438,-0.805415,0.460969,0.001547,0.577053,-0.244203,-0.502610,-1.040643,-0.027078,-0.147346,0.867743,-0.286161,-0.114783,0.161768,165.01,0
101620,67895.0,-0.326965,-0.740525,2.818067,0.291949,-1.376193,0.512689,-1.489943,0.476907,0.041457,0.300602,-0.788864,-0.581301,-0.582155,-1.185207,-0.392105,-0.553832,1.939349,-1.454913,2.291508,0.297547,0.335559,1.287401,-0.027948,0.482511,-0.629584,0.126273,0.265631,0.175552,1.00,0
249842,154604.0,2.192563,-0.611910,-1.301295,-0.435089,-0.611411,-1.302581,-0.240102,-0.395806,-0.575667,0.991762,-1.033266,-0.371114,-0.021130,0.346330,0.521964,-1.432750,-0.214065,1.121804,-0.967386,-0.626831,-0.158442,0.111351,0.123754,-0.002083,-0.060210,0.791314,-0.067378,-0.066303,9.00,0
261510,160050.0,-0.907526,0.797012,-1.208129,-1.903143,2.760956,4.103260,-0.867687,-1.713287,-0.029723,-1.345352,-0.638324,0.353489,-0.709293,0.549074,-0.562886,-0.584688,-0.108541,-0.354650,-0.153213,0.461290,-1.072390,1.066609,0.038155,0.740876,-0.159923,0.106470,0.018929,0.180719,1.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,-5.587794,2.115795,-5.417424,-1.235123,-6.665177,0.401701,-2.897825,-4.570529,-1.315147,0.391167,1.252967,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,-3.232153,2.858466,-3.096915,-0.792532,-5.210141,-0.613803,-2.155297,-3.267116,-0.688505,0.737657,0.226138,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,-3.463891,1.794969,-2.775022,-0.418950,-4.057162,-0.712616,-1.603015,-5.035326,-0.507000,0.266272,0.247968,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,-5.245984,1.933520,-5.030465,-1.127455,-6.416628,0.141237,-2.549498,-4.614717,-1.478138,-0.035480,0.306271,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


In [173]:
new_dataset['Class'].value_counts()

Class
0    492
1    492
Name: count, dtype: int64

In [174]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,97497.042683,-0.023831,0.138689,-0.050222,-0.009750,0.028808,0.049448,0.029406,0.076829,0.053291,-0.006167,0.054344,-0.019322,-0.038044,-0.035994,0.036173,0.054845,0.065312,0.034730,-0.010797,0.042606,-0.004893,0.011715,-0.024848,-0.032245,0.006227,-0.004942,0.027544,0.008617,85.242520
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [175]:
# splitting the data into features and targets
X = new_dataset.drop(columns='Class', axis=1)
Y = new_dataset['Class']

In [176]:
X

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
65874,51782.0,0.985125,-1.735421,-0.347965,-1.194368,-1.362408,-0.937011,-0.150100,-0.414604,-2.173645,1.295732,-0.646338,-0.786700,0.765736,-0.090143,0.398949,-0.438219,0.486537,-0.425044,0.073022,0.298054,-0.404756,-1.506348,-0.100852,-0.015105,0.182456,-0.539072,-0.046081,0.068398,313.43
255857,157434.0,-1.521088,-1.708103,0.906377,-2.766646,0.234871,0.579226,-0.065909,0.161328,-1.851499,0.432489,-0.708131,-0.537107,-0.147410,-0.848400,-3.137438,-0.805415,0.460969,0.001547,0.577053,-0.244203,-0.502610,-1.040643,-0.027078,-0.147346,0.867743,-0.286161,-0.114783,0.161768,165.01
101620,67895.0,-0.326965,-0.740525,2.818067,0.291949,-1.376193,0.512689,-1.489943,0.476907,0.041457,0.300602,-0.788864,-0.581301,-0.582155,-1.185207,-0.392105,-0.553832,1.939349,-1.454913,2.291508,0.297547,0.335559,1.287401,-0.027948,0.482511,-0.629584,0.126273,0.265631,0.175552,1.00
249842,154604.0,2.192563,-0.611910,-1.301295,-0.435089,-0.611411,-1.302581,-0.240102,-0.395806,-0.575667,0.991762,-1.033266,-0.371114,-0.021130,0.346330,0.521964,-1.432750,-0.214065,1.121804,-0.967386,-0.626831,-0.158442,0.111351,0.123754,-0.002083,-0.060210,0.791314,-0.067378,-0.066303,9.00
261510,160050.0,-0.907526,0.797012,-1.208129,-1.903143,2.760956,4.103260,-0.867687,-1.713287,-0.029723,-1.345352,-0.638324,0.353489,-0.709293,0.549074,-0.562886,-0.584688,-0.108541,-0.354650,-0.153213,0.461290,-1.072390,1.066609,0.038155,0.740876,-0.159923,0.106470,0.018929,0.180719,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,-5.587794,2.115795,-5.417424,-1.235123,-6.665177,0.401701,-2.897825,-4.570529,-1.315147,0.391167,1.252967,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,-3.232153,2.858466,-3.096915,-0.792532,-5.210141,-0.613803,-2.155297,-3.267116,-0.688505,0.737657,0.226138,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,-3.463891,1.794969,-2.775022,-0.418950,-4.057162,-0.712616,-1.603015,-5.035326,-0.507000,0.266272,0.247968,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,-5.245984,1.933520,-5.030465,-1.127455,-6.416628,0.141237,-2.549498,-4.614717,-1.478138,-0.035480,0.306271,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00


In [177]:
Y

65874     0
255857    0
101620    0
249842    0
261510    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64

In [178]:
# split the data into training data and testing data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)
print(X.shape, X_train.shape, X_test.shape)

(984, 30) (787, 30) (197, 30)


--> MODEL TRAINING

In [179]:
# model training
# logistic regression
model = LogisticRegression()

In [180]:
# training the logistic regression model with training data
model.fit(X_train, Y_train)
# model evaluation
# accuracy score
# accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Accuracy on training data : ', training_data_accuracy)
# accuracy on testing data
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy on testing data : ', testing_data_accuracy)

Accuracy on training data :  0.9440914866581956
Accuracy on testing data :  0.934010152284264


c:\Users\dibye\OneDrive\Desktop\Credit Card Fraud Detection\CCFD_ENV\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [181]:
# # model improvement
# # taking max features 3000
# model = LogisticRegression(max_iter=3000)

# model.fit(X_train, Y_train)

# X_train_prediction = model.predict(X_train)
# training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
# print('Accuracy on training data : ', training_data_accuracy)

# X_test_prediction = model.predict(X_test)
# testing_data_accuracy = accuracy_score(X_test_prediction, Y_test)
# print('Accuracy on testing data : ', testing_data_accuracy)

In [182]:
# from sklearn.ensemble import BaggingClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score


# lr = LogisticRegression(solver='saga', class_weight='balanced', max_iter=1000)


# bagging_model = BaggingClassifier(
#     estimator=lr,           
#     n_estimators=10,
#     max_samples=0.8,
#     max_features=1.0,
#     random_state=42
# )


# bagging_model.fit(X_train, Y_train)


# train_pred = bagging_model.predict(X_train)
# test_pred = bagging_model.predict(X_test)

# print("Training Accuracy:", accuracy_score(Y_train, train_pred))
# print("Testing Accuracy:", accuracy_score(Y_test, test_pred))


In [183]:
# from sklearn.ensemble import VotingClassifier
# from sklearn.ensemble import RandomForestClassifier
# import xgboost as xgb


# lr = LogisticRegression(solver='saga', class_weight='balanced', max_iter=1000)
# rf = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
# xg = xgb.XGBClassifier(scale_pos_weight=(len(Y_train)-sum(Y_train))/sum(Y_train), use_label_encoder=False, eval_metric='logloss')


# voting_model = VotingClassifier(
#     estimators=[('lr', lr), ('rf', rf), ('xg', xg)],
#     voting='soft'   
# )


# voting_model.fit(X_train, Y_train)


# train_pred = voting_model.predict(X_train)
# test_pred = voting_model.predict(X_test)

# print("Training Accuracy:", accuracy_score(Y_train, train_pred))
# print("Testing Accuracy:", accuracy_score(Y_test, test_pred))


In [184]:
# from sklearn.ensemble import StackingClassifier

# base_models = [
#     ('lr', LogisticRegression(solver='saga', class_weight='balanced', max_iter=1000)),
#     ('rf', RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)),
#     ('xg', xgb.XGBClassifier(scale_pos_weight=(len(Y_train)-sum(Y_train))/sum(Y_train), use_label_encoder=False, eval_metric='logloss'))
# ]

# stack_model = StackingClassifier(
#     estimators=base_models,
#     final_estimator=LogisticRegression()
# )


# stack_model.fit(X_train, Y_train)

# train_pred = stack_model.predict(X_train)
# test_pred = stack_model.predict(X_test)

# print("Training Accuracy:", accuracy_score(Y_train, train_pred))
# print("Testing Accuracy:", accuracy_score(Y_test, test_pred))


In [185]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.optimizers import Adam

# # Build a simple MLP
# model = Sequential()
# model.add(Dense(16, input_dim=X_train.shape[1], activation='relu'))
# model.add(Dense(1, activation='sigmoid')) 


# model.compile(optimizer=Adam(0.001), 
#               loss='binary_crossentropy', 
#               metrics=['accuracy'])


# history = model.fit(X_train, Y_train, epochs=50, batch_size=32, validation_split=0.2)

# train_loss, train_acc = model.evaluate(X_train, Y_train, verbose=0)
# print("Training Accuracy:", train_acc)


# test_loss, test_acc = model.evaluate(X_test, Y_test, verbose=0)
# print("Testing Accuracy:", test_acc)


### GET BEST ACCURACY ON LOGISTIC ALGO AND MAX FEATURES method